In [1]:
import os
os.chdir('../')

## Observe variation in the observation space

In [2]:
import json

class MockAIGame:
    observations = []
    rewards = []
    env = None

    def __init__(self, env):
        self.env = env
    
config = None
with open('config.json') as f:
    config = json.load(f)


In [3]:
import gym

all_observations = []

In [4]:

env = gym.make('CartPole-v1')

for i_episode in range(200):
    observation = env.reset()
    for t in range(1000):
        env.render()
#         print(observation)
        all_observations.append(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done:
#             print("Episode finished after {} timesteps".format(t+1))
            break

env.close()


In [5]:
len(all_observations)

4523

In [6]:
import pandas as pd

cols = ['pos', 'velocity', 'angle', 'angular velocity']
df = pd.DataFrame(data=all_observations, columns=cols)

In [7]:
df.describe()

,pos,velocity,angle,angular velocity
count,4523.000000,4523.000000,4523.000000,4523.000000
mean,-0.009191,-0.019753,-0.009521,-0.017292
std,0.099260,0.528773,0.090523,0.783163
min,-0.762246,-2.285626,-0.209095,-2.849260
25%,-0.043869,-0.370242,-0.064795,-0.538533
50%,-0.005464,-0.008009,-0.011314,-0.012012
75%,0.035057,0.347880,0.041752,0.479990
max,0.539514,2.653481,0.209206,2.632457


In [8]:
print('theoretical min:', env.observation_space.low)
print('theoretical max:', env.observation_space.high)

theoretical min: [-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]
theoretical max: [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]


## Analyze game inputs on those observations

In [9]:
from game_interface import GameInterface

game = MockAIGame(env)
gi = GameInterface(game, config)

In [10]:
all_firing_rates = []

for obs in all_observations:
    game.observations = [obs]
    all_firing_rates.append(gi.input_firing_rates()[:4])
    
print(len(all_firing_rates))

4523


In [11]:
df_fr = pd.DataFrame(data=all_firing_rates, columns=['fr_'+c for c in cols])

In [12]:
df_fr.describe()

,fr_pos,fr_velocity,fr_angle,fr_angular velocity
count,4523.000000,4523.000000,4523.000000,4523.000000
mean,74.856383,74.315639,73.295183,74.350244
std,1.550939,18.435506,16.208143,25.763694
min,63.089905,13.848065,37.561694,8.207936
25%,74.314552,61.272396,63.398501,55.279349
50%,74.914624,74.699663,72.974250,74.549537
75%,75.547764,87.915497,82.475708,92.661821
max,83.429899,140.133779,112.458235,139.938219


In [13]:
maxFiring = config['net']['InputMaxRate']
df_interval = pd.DataFrame(data=[[maxFiring/fr for fr in k]for k in all_firing_rates], columns=['int_'+c for c in cols])

In [14]:
df_interval.describe()

,int_pos,int_velocity,int_angle,int_angular velocity
count,4523.000000,4523.000000,4523.000000,4523.000000
mean,2.004750,2.175908,2.159594,2.377400
std,0.044198,0.708282,0.536902,1.249285
min,1.797917,1.070406,1.333829,1.071902
25%,1.985499,1.706184,1.818717,1.618790
50%,2.002279,2.008041,2.055520,2.012085
75%,2.018447,2.448084,2.365987,2.713491
max,2.377559,10.831838,3.993430,18.274996
